![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

# Concatenating and Merging Dataframes

- https://pandas.pydata.org/pandas-docs/stable/merging.html

In this lesson we will use 3 datasets provided by Yelp.
* yelp_business
* yelp_business_hours
* yelp_reviews

In [0]:
import pandas as pd
import numpy  as np
import psycopg2 as ps

In [0]:
# Connect to database
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
port = '5432'
# TODO Write your username and database password in the following two variables
user_name = '' #enter username
pwd = '' #enter your database password found in the profile tab in Strata Scratch

try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

Connected!


In [0]:
# Get the data we will use for this exercise

def get_dataset(dataset_name):
    #Write SQL below to pull datasets 
    cur = conn.cursor()
    cur.execute(""" 
                SELECT *  FROM datasets.{0}; 
                """.format(dataset_name))
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description] 
    conn.commit()

    #create the pandas dataframe
    dataframe = pd.DataFrame(data, columns=colnames)

    #close the connection
    cur.close()
    
    return dataframe

yelp_business = get_dataset("yelp_business")
yelp_hours    = get_dataset("yelp_business_hours")
yelp_reviews  = get_dataset("yelp_reviews")

# a little bit preprocessing
yelp_reviews['business_name'] = yelp_reviews['business_name'].apply(lambda x: '"{0}"'.format(x))

In [0]:
yelp_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [0]:
yelp_hours.head()

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
1,He-G7vWjzVUysIKrfNbPUQ,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
2,KQPW8lFf1y5BT2MxiSZ3QA,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
4,PfOCPjBrlQAnz__NXj9h_w,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0


In [0]:
yelp_reviews.head()

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
0,"""Thai Pan Fresh Exotic Cuisine""",0ESSqLfOae77muWTv_zUqA,r-t7IiTSD0QZdt8lOUCqeQ,3,2011-02-11,Lately i have been feeling homesick for asian ...,1,1,1
1,"""Salon Lola""",VyAKIaj_Rmsf_ZCHcGJyUw,SS85hfTApRnbTPcJadra8A,5,2010-05-30,I love Marilo! She understands my hair type a...,0,0,0
2,"""Chipotle Mexican Grill""",DusrkpkTGPGkqK13xO1TZg,xAVu2pZ6nIvkdHh8vGs84Q,3,2011-11-26,Standard Chipotle fare - consistently good; no...,0,0,0
3,"""Sam's Deli""",A5FL-YYXnr2hnvXQbgkkrQ,Xuvj2Fq6c3mCmplVG7h21w,4,2013-10-10,Very good prices with quality ingredients.,0,0,0
4,"""Petco""",SzhIWfzLHUZfiAaVTtKHxw,jHH_UDpCnF-YTnveyoMTIA,1,2012-09-02,I went to this Petco a few times and was a bit...,0,1,0


## Concatenatations

### What are they?

If you think of your dataframes as rectangles in a 2D coordinate system concatenating means taking two or more rectangles and putting them next to each other so they touch on the concatenating axis.

We will use two types of concatenations, numpy and pandas.

The pandas version builds upon the numpy basics by incorporating ideas like indexes and keys but the main idea is the same.

The functions we will learn first are
- [np.hstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dstack.html)
- [np.vstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html)

In numpy lingo stack means concatenate. Pandas uses the terminology concat.

In [0]:
array1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

array1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [0]:
array2 = np.array([[1, 0, 1], [1, 1, 1], [0, 1, 0]])

array2

array([[1, 0, 1],
       [1, 1, 1],
       [0, 1, 0]])

In [0]:
# hstack stands for horizontal stack (stack axis is parallel to Y axis)
print(np.hstack([array1, array2]))

[[1 2 3 1 0 1]
 [4 5 6 1 1 1]
 [7 8 9 0 1 0]]


In [0]:
# vstack stands for vertical stack (stack axis is parallel to X axis)
print(np.vstack([array1, array2]))

[[1 2 3]
 [4 5 6]
 [7 8 9]
 [1 0 1]
 [1 1 1]
 [0 1 0]]


Both stacking functions take a list of nd-arrays and return a single nd-array.

Here is another example this time using 1d-arrays and stacking to build a 2d-array.

In [0]:
row1 = np.array([3, -3, 3])
row2 = np.array([6, -9, 27])
row3 = np.array([9, -27, 81])

np.vstack([row1, row2, row3])

array([[  3,  -3,   3],
       [  6,  -9,  27],
       [  9, -27,  81]])

### Concatenating dataframes

Because dataframes are a bit more than 2d-arrays we can concatenate them as well.

We can use the numpy functions `hstack` and `vstack` but it is recommended to use `pd.concatenate` because of indexes.
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

There are a lot of arguments but as always let's start small.

In [0]:
df1 = pd.DataFrame({
    "name": ["A", "B", "C", "D"],
    "age": [22, 18, 46, 81],
}, index=[0, 1, 2, 3])

df2 = pd.DataFrame({
    "name": ["G", "H", "I"],
    "age": [10, 20, 30]
}, index=[4, 5, 6])

print(df1)
print(df2)

  name  age
0    A   22
1    B   18
2    C   46
3    D   81
  name  age
4    G   10
5    H   20
6    I   30


In [0]:
# simplest possible use case will do a vstack
pd.concat([df1, df2])

,name,age
0,A,22
1,B,18
2,C,46
3,D,81
4,G,10
5,H,20
6,I,30


In [0]:
# Passing axis=1 will do hstack
pd.concat([df1, df2], axis=1)

,name,age,name,age
0,A,22.0,NaN,NaN
1,B,18.0,NaN,NaN
2,C,46.0,NaN,NaN
3,D,81.0,NaN,NaN
4,NaN,NaN,G,10.0
5,NaN,NaN,H,20.0
6,NaN,NaN,I,30.0


**Quick remark: np.hstack and np.vstack will throw an error if shapes do not match, pd.concatenate will extend the dataframes with null values depending on the join parameter.**

Passing the keys property will make the resulting dataframe have a [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) whose first level is the value which describes the dataframe.

In [0]:
pd.concat([df1, df2], keys=['df_left', 'df_right'])

name  age
df_left  0    A   22
         1    B   18
         2    C   46
         3    D   81
df_right 4    G   10
         5    H   20
         6    I   30

When you concatenate on axis where the shapes do not match you should set the join paramater to one of its two values:
- "inner" which means take the intersection over the index axis of the dataframes being concatenated
- "outer" which means take the union over the index axis of the dataframes being concatenated

The default value is "outer" which gives the behaviour as described by the quick remark above.

In [0]:
pd.concat([df1, df2], axis=1, join='inner')

,name,age,name,age


In [0]:
pd.concat([df1, df2], axis=1, join='outer')

,name,age,name,age
0,A,22.0,NaN,NaN
1,B,18.0,NaN,NaN
2,C,46.0,NaN,NaN
3,D,81.0,NaN,NaN
4,NaN,NaN,G,10.0
5,NaN,NaN,H,20.0
6,NaN,NaN,I,30.0


The final important parameter to talk about is `ignore_index`.

If you pass `ignore_index=True` pandas will concatenate the dataframes without looking at the index.

Generally you do not need to use this but if you concatenate dataframes with different index types you will need this.

The [append](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-using-append) method is similar to concat in the sense that the following holds true

`df1.append(df2) = pd.concat([df1, df2])`

You should prefer concat almost always because it is more general.

#### Practical example 

List comprehension and pd.concat to get 1, 3, 5 stared rows from yelp_reviews dataset.

In [0]:
pd.concat([
           yelp_reviews[yelp_reviews.stars == stars] 
           for stars in ['1', '3', '5']
          ])

# another solution without using pd.concat
# yelp_reviews[yelp_reviews.stars.isin(['1', '3', '5'])]

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
4,"""Petco""",SzhIWfzLHUZfiAaVTtKHxw,jHH_UDpCnF-YTnveyoMTIA,1,2012-09-02,I went to this Petco a few times and was a bit...,0,1,0
12,"""Canteen Modern Tequila Bar""",qnpnnZ_JrOLsIdtn5aB9Hw,_4lqpCYCqOQzbB6xQGGhrQ,1,2010-10-14,"Ok, so the grand opening hasn't even happened ...",1,3,1
60,"""Fairmont Scottsdale Princess""",pmXF1ajhiMietpARXXX-iw,PBdpLP3yMHV79Y7EjuBY4g,1,2010-11-25,We went for the Thanksgiving Brunch at the LV ...,0,0,0
71,"""Gordon Biersch Brewery Restaurant""",avwF0OJFhNxu-s95JDhS7A,ZAXpsNT5fi4csd86AXVqPw,1,2013-09-15,Got food poisoning off a simple Caesar salad. ...,0,2,0
79,"""Golden Buddha""",I7wxA8sjH9B5_4tTr5cNyQ,Z3qVl-1JgYnAe4py6VC5ug,1,2010-01-09,"First of all, I visited this place many times ...",0,1,0
93,"""A Better Grooming""",SQe0XmT8swBPiKaf31w7kw,VnOtsJ2514YIAd0fM2O8oQ,1,2011-05-14,I have had two bad experiences. Call me stupid...,0,0,0
104,"""Paradise Bakery and Cafe""",yy2DcWTRcBIn--zXD6nmOw,tA9EwCwdYw3sAMxvrONvaA,1,2012-12-04,If there was an option for no stars I would ha...,0,0,0
108,"""Pork on a Fork""",g0y3fFNF0uHC4MNGOqAM2w,sxc1NzZ3pJKUfHquGkF_bQ,1,2013-10-29,So disappointing. The most disappointing part ...,0,0,0
142,"""Meineke Car Care Center""",Yh3X-ySSbYRBC8aoArIkAg,YloWz4Nwfx1qDH-BX1KIFw,1,2011-07-23,I attempted to get my air conditioning service...,0,1,0
144,"""Joe's Crab Shack""",MV802Gg5DiZFJ0QoJi2zAQ,74Tpb9Lz4SBAQ8AB6vJQ_g,1,2013-10-06,Went inside with my boyfriend around 10pm on a...,0,1,0


## Merges and Joins
- https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

pandas can perform relational joins in memory using the `merge` method which we discuss for the remainder of this lesson.

Each dataframe also has a `join` method but the following holds true (for details see the docs)

`df1.join(df2) = pd.merge(df1, df2, left_index=True, right_index=True, how="left")`

By default merge does an inner join, while [`pd.DataFrame.join`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) does a left join but these are configurable with the how parameter.

The most imporant parameter of `pd.merge` is the choice of join type in the parameter called `how`.
- "inner" which corresponds to INNER JOIN in SQL
- "outer" which corresponds to FULL OUTER JOIN in SQL
- "left" which corresponds to LEFT OUTER JOIN in SQL
- "right" which corresponds to RIGHT OUTER JOIN in SQL

Second most important choice is what to join on.
- Column or a list of columns (e.g. ["bussiness_id"])
- Index 

You can choose either of them for both left and right(e.g. columns for left and index for right, or columns for both or indices for both)

*By default merge will work with indexes*

#### Example 1 of merge function

An example of inner join between yelp_business and yelp_hours on business_id column.

In [0]:
pd.merge(yelp_business, yelp_hours, how="inner", left_on=["business_id"], right_on=["business_id"])

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",None,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,None
6,kCoE3jvEtg6UVz5SOD3GVw,"""BDJ Realty""",Summerlin,"""2620 Regatta Dr, Ste 102""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,5,1,Real Estate Services;Real Estate;Home Services...,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,None,None
7,OD2hnuuTJI9uotcKycxg1A,"""Soccer Zone""",None,"""7240 W Lake Mead Blvd, Ste 4""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,9,1,Shopping;Sporting Goods,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,10:0-18:0,11:0-16:0
8,EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",None,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food,None,None,None,None,None,None,None
9,TGWhGNusxyMaA4kQVBNeew,"""Detailing Gone Mobile""",None,"""""",Henderson,NV,89014,36.055825,-115.046350,5.0,7,1,Automotive;Auto Detailing,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0


#### Example 2 of merge function

If the join columns are named the same in both dataframes you can use the `on` parameter and write less code.

The result of this join and the join right above is the same.

In [0]:
pd.merge(yelp_business, yelp_hours, how="inner", on=["business_id"])

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",None,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,None
6,kCoE3jvEtg6UVz5SOD3GVw,"""BDJ Realty""",Summerlin,"""2620 Regatta Dr, Ste 102""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,5,1,Real Estate Services;Real Estate;Home Services...,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,None,None
7,OD2hnuuTJI9uotcKycxg1A,"""Soccer Zone""",None,"""7240 W Lake Mead Blvd, Ste 4""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,9,1,Shopping;Sporting Goods,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,10:0-18:0,11:0-16:0
8,EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",None,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food,None,None,None,None,None,None,None
9,TGWhGNusxyMaA4kQVBNeew,"""Detailing Gone Mobile""",None,"""""",Henderson,NV,89014,36.055825,-115.046350,5.0,7,1,Automotive;Auto Detailing,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0


#### Example 3 of merge function

You can also join on indexes.

First we must make the bussiness_id column the index in both dataframes (see [set_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html) for more info).

Then we join on index.

In [0]:
pd.merge(left=yelp_business.set_index(["business_id"]),
         right=yelp_hours.set_index(["business_id"]),
         how="inner",
         left_index=True,
         right_index=True)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0
o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",None,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,None
kCoE3jvEtg6UVz5SOD3GVw,"""BDJ Realty""",Summerlin,"""2620 Regatta Dr, Ste 102""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,5,1,Real Estate Services;Real Estate;Home Services...,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,None,None
OD2hnuuTJI9uotcKycxg1A,"""Soccer Zone""",None,"""7240 W Lake Mead Blvd, Ste 4""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,9,1,Shopping;Sporting Goods,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,10:0-18:0,11:0-16:0
EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",None,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food,None,None,None,None,None,None,None


#### The suffixes parameter

The `suffixes` parameter is a tuple made of two strings.
- The first element of this tuple is the suffix for columns from the left dataframe
- The second element of this tuple is the suffix for columns from the right dataframe

The suffix is applied only to columns which have identical names and are not join keys.

For example
- d1 has columns ["key_id", "first_name", "age"]
- d2 has columns ["key_id", "last_name", "age"]

`pd.merge(d1, d2, on="key_id", suffixes=("_D1", "_D2"))`

The resulting dataframe will have the following columns
- ["key_id", "first_name", "last_name", "age_D1", "age_D2"]

Because age is present in both data frames and we must preserve both columns the solution is to rename both of them by appending a suffix string.

The default suffixes are ("_x", "_y") so you don't have to pass this paramater.

For no good reason other than to trouble you dear reader the `pd.DataFrame.join` method does not have a suffixes paramater but two paramaters:
- lsuffix which stands for left suffix
- rsuffix which stands for right suffix

The idea is still the same though.

#### Example 4. Full outer join in pandas.

Because this a full outer join all join keys are present (letters A to I).

In [0]:
pd.merge(df1, df2, how="outer", on=["name"])

,name,age_x,age_y
0,A,22.0,NaN
1,B,18.0,NaN
2,C,46.0,NaN
3,D,81.0,NaN
4,G,NaN,10.0
5,H,NaN,20.0
6,I,NaN,30.0


#### Example 5. Left outer join in pandas.

Because this a left outer join only keys from the left dataframe are present (A, B, C, D).

In [0]:
pd.merge(df1, df2, how="left", on=["name"], suffixes=("_df1", "_df2"))

,name,age_df1,age_df2
0,A,22,NaN
1,B,18,NaN
2,C,46,NaN
3,D,81,NaN


#### Example 1 of using the join function

Here we use a left outer join because we want a list of all bussiness irregardles if they have an entry in the hours table. If they don't have an entry all columns (Monday to Sunday) will be set to null which is the semantics of left join.

In [0]:
yelp_business.set_index("business_id", inplace=True)

yelp_hours.set_index("business_id", inplace=True)

yelp_business.join(yelp_hours)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0
o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",None,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,18:0-0:0,None
kCoE3jvEtg6UVz5SOD3GVw,"""BDJ Realty""",Summerlin,"""2620 Regatta Dr, Ste 102""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,5,1,Real Estate Services;Real Estate;Home Services...,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,None,None
OD2hnuuTJI9uotcKycxg1A,"""Soccer Zone""",None,"""7240 W Lake Mead Blvd, Ste 4""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,9,1,Shopping;Sporting Goods,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,11:0-19:0,10:0-18:0,11:0-16:0
EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",None,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food,None,None,None,None,None,None,None


#### Example 2 of using the join function.

Using left outer join where the right dataframe is an empty dataframe you can add a lot of new columns to your existing dataframe by default filled with NaN.

Here we would like to add "address_street_number" and "address_street" to yelp_business.

This is not very useful here because we have little columns but this combined with `pd.DataFrame.fillna` method could make coding some algorithms an easier endeavour.

In [0]:
dummy_empty_df = pd.DataFrame(columns=["address_street_number", "address_street"])

yelp_business.join(dummy_empty_df)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,address_street_number,address_street
business_id,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",None,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,NaN,NaN
He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",None,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,NaN,NaN
KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",None,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,NaN,NaN
8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",None,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,NaN,NaN
PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",None,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,NaN,NaN
o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",None,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,NaN,NaN
kCoE3jvEtg6UVz5SOD3GVw,"""BDJ Realty""",Summerlin,"""2620 Regatta Dr, Ste 102""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,5,1,Real Estate Services;Real Estate;Home Services...,NaN,NaN
OD2hnuuTJI9uotcKycxg1A,"""Soccer Zone""",None,"""7240 W Lake Mead Blvd, Ste 4""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,9,1,Shopping;Sporting Goods,NaN,NaN
EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",None,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food,NaN,NaN


## Exercises

#### Exercise #1

Find all bussiness which are from 'AZ', 'PA', 'OH', 'BW', 'NV' using concatenation.

#### Exercise #2

Find all bussiness which are from 'AZ', 'PA', 'OH', 'BW', 'NV' using inner join.

Hint: You can use a temporary dataframe.

#### Exercise #3

Make one giant table from yelp_business, yelp_reviews, yelp_business_hours.

Hint: use the merge function

#### Exercise #4

Find the maximal review count from yelp_business and then using inner join find the business (or business) which have that review count.

#### Exercise #5

The yelp_reviews dataset has 14 rows where the stars column has a value of '?'.

We want to explore the businesses these reviews relate to.